In [14]:
import os
from dotenv import load_dotenv
load_dotenv()

True

In [15]:
# 
PINECONE_API_KEY = os.getenv("PINECONE_API_KEY")
os.environ["HF_TOKEN"]=os.getenv("HF_TOKEN")

In [16]:
from langchain_community.retrievers import PineconeHybridSearchRetriever

In [17]:
from pinecone import Pinecone,ServerlessSpec
index_name = "hybrid-search-with-pinecone"  # index name must be in smalll case
# intialize pinecone client
pc = Pinecone(api_key=PINECONE_API_KEY)

# create index
if index_name not in pc.list_indexes().names():
    pc.create_index(name=index_name,
                    dimension=384,
                    metric="dotproduct",
                    spec=ServerlessSpec(cloud="aws",region="us-east-1"),)
    print(f"Index {index_name} created successfully")

In [20]:
index = pc.Index(index_name)
index

In [21]:
from langchain_huggingface import HuggingFaceEmbeddings
embeddings = HuggingFaceEmbeddings(model_name="all-MiniLM-L6-v2")
embeddings

HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False)

In [22]:
from pinecone_text.sparse import BM25Encoder

bm25_encoder = BM25Encoder().default()
bm25_encoder

In [23]:
sentences = [
    "In 2023, I visited paris",
    "In 2022, I visited New York",
    "In 2021, I visited London",
]

In [24]:
## applying bm25 encoder which is using TF-IDF to convert sentences to vectors
bm25_encoder.fit(sentences)

## store the values to a json file
bm25_encoder.dump("bm25_encoder.json")

## load the values from the json file
loaded_bm25_encoder = BM25Encoder().load("bm25_encoder.json")

100%|██████████| 3/3 [00:00<00:00, 83.26it/s]


In [25]:
retriever = PineconeHybridSearchRetriever(embeddings=embeddings, index=index, sparse_encoder=loaded_bm25_encoder)

In [26]:
retriever

PineconeHybridSearchRetriever(embeddings=HuggingFaceEmbeddings(model_name='all-MiniLM-L6-v2', cache_folder=None, model_kwargs={}, encode_kwargs={}, multi_process=False, show_progress=False), sparse_encoder=<pinecone_text.sparse.bm25_encoder.BM25Encoder object at 0x000002B634793D60>, index=<pinecone.data.index.Index object at 0x000002B6347910C0>)

In [27]:
retriever.add_texts(sentences)

100%|██████████| 1/1 [00:01<00:00,  1.85s/it]
